# Build a Dashboard Application with Plotly Dash


In [1]:
dash_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

In [2]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv(dash_url)
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
launch_site_names = spacex_df['Launch Site'].unique()

# Create a dash application
app = dash.Dash(__name__)
options = [{'label': 'All Sites', 'value': 'ALL'}] + [{'label': site, 'value': site} for site in launch_site_names]

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # Define the dcc.Dropdown component
                                dcc.Dropdown(
                                    id='site-dropdown',      # ID of the dropdown component
                                    options=options,         # List of options for the dropdown menu
                                    value='ALL',             # Default dropdown value
                                    placeholder="Select a Launch Site here",  # Placeholder text for the dropdown
                                    searchable=True          # Enable search functionality
                                ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(
                                    id='payload-slider',
                                    min=0,
                                    max=10000,
                                    step=1000,
                                    marks={0: '0', 1000: '1000', 2000: '2000', 3000: '3000', 4000: '4000', 5000: '5000', 6000: '6000', 7000: '7000', 8000: '8000', 9000: '9000', 10000: '10000'},
                                    value=[min_payload, max_payload]
                                ),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_chart(entered_site):
    # Filter the dataframe based on the selected site
    if entered_site == 'ALL':
        filtered_df = spacex_df
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
    
    # Calculate the count of success (class=1) and failure (class=0) launches
    success_count = filtered_df[filtered_df['class'] == 1]['class'].count()
    failure_count = filtered_df[filtered_df['class'] == 0]['class'].count()
    
    # Create a pie chart to visualize the success and failure counts
    labels = ['Success', 'Failure']
    values = [success_count, failure_count]
    fig = px.pie(names=labels, values=values, title='Success/Failure Launches')
    
    return fig
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

# Define the callback function
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")]
)
def update_scatter_chart(selected_site, selected_payload_range):
    filtered_df = spacex_df
    if selected_site != 'ALL':
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
    
    min_payload, max_payload = selected_payload_range
    filtered_df = filtered_df[(filtered_df['Payload Mass (kg)'] >= min_payload) & (filtered_df['Payload Mass (kg)'] <= max_payload)]
    fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version', title='Payload Mass vs Launch Outcome')

    return fig




C:\Users\vasil\AppData\Local\Temp\ipykernel_20676\85012453.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\vasil\AppData\Local\Temp\ipykernel_20676\85012453.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
